## 4. PyLipid Full Dataset Joining Script

In [39]:
import pandas as pd
import numpy as np
import re

In [40]:
trpv = 1
miss_res = 114


lipids = [lipid for lipid in "CHOL.POPC.PAPC.POPE.DIPE.PAPS.DPSM.POPA.PAP1.PAP6".split(".")]
folder = f"/home/apmlab/Projects/Mario/TRPVs/Simulations/TRPV{trpv}/PyLipid"


#HI
aminoacids = [aa for aa in "ALA.CYS.ASP.GLU.PHE.GLY.HIS.ILE.LYS.LEU.MET.ASN.PRO.GLN.ARG.SER.THR.VAL.TRP.TYR".split(".")]
hydro_index = [1.8, 2.5, -3.5, -3.5, 2.8, -0.4, -3.2, 4.5, -3.9, 3.8, 1.9, -3.5, -1.6, -3.5, -4.5, -0.8, -0.7, 4.2, -0.9, -1.3]
aa_hi ={aa:hi for aa, hi in zip(aminoacids, hydro_index)}


In [41]:
lipid_datasets = {lipid:pd.read_csv(f"{folder}/Interaction_{lipid}/Dataset_{lipid}/Dataset.csv")\
                   for lipid in lipids}

for lipid, dataset in lipid_datasets.items():
    print(lipid)
    lipid_datasets[lipid].loc[:, "Lipid"] = lipid #Assign lipid
    lipid_datasets[lipid].loc[:, "aa"] = [re.match(r"([0-9]*)([A-Z]{3})", aa)[2] for aa in dataset["Residue"]] #Assign aa name
    lipid_datasets[lipid].loc[:, "HI"] = [aa_hi[aa] for aa in dataset["aa"]]

    #Weighted HI normalized by max restime (5)
    lipid_datasets[lipid].loc[:, "Weighted HI"] = dataset["HI"] * dataset["Residence Time"] / 5

    #Assign pdb number
    pdb_nums = [int(re.match(r"([0-9]*)([A-Z]{3})", res)[1]) + miss_res\
                 for res in lipid_datasets[lipid].loc[:, "Residue"]]
    lipid_datasets[lipid].loc[:, "pdb_resnum"] =  pdb_nums

    #For TRPV1 loop
    if trpv == 1:
        lipid_datasets[lipid].loc[lipid_datasets[lipid]["pdb_resnum"] > 601, "pdb_resnum"] =\
        lipid_datasets[lipid].loc[lipid_datasets[lipid]["pdb_resnum"] > 601, "pdb_resnum"] + 25

    #Filter by Restime
    if "Binding Site Residence Time" in dataset.columns:
        lipid_datasets[lipid] = dataset.loc[dataset["Binding Site Residence Time"] >= 2.5, :]

    #Assign number of aa per Binding Site ID
    #Assign also GRAVY score
    if "Binding Site ID" in lipid_datasets[lipid].columns:
        for bsid in set(lipid_datasets[lipid]["Binding Site ID"]):
            #Number of aa
            num_aa = int(len(lipid_datasets[lipid].loc[dataset["Binding Site ID"] == bsid, "Residue"]))
            lipid_datasets[lipid].loc[lipid_datasets[lipid]["Binding Site ID"] == bsid, "Binding Site Number aa"] = num_aa
    
            #GRAVY score
            gravy_score = lipid_datasets[lipid].loc[lipid_datasets[lipid]["Binding Site ID"] == bsid, "Weighted HI"].mean()
            lipid_datasets[lipid].loc[lipid_datasets[lipid]["Binding Site ID"] == bsid, "GRAVY_score"] = gravy_score


    #Save Excel
  

CHOL
POPC
PAPC
POPE
DIPE
PAPS
DPSM
POPA
PAP1
PAP6


/tmp/ipykernel_749390/1164419338.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lipid_datasets[lipid].loc[lipid_datasets[lipid]["Binding Site ID"] == bsid, "Binding Site Number aa"] = num_aa
/tmp/ipykernel_749390/1164419338.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lipid_datasets[lipid].loc[lipid_datasets[lipid]["Binding Site ID"] == bsid, "GRAVY_score"] = gravy_score
/tmp/ipykernel_749390/1164419338.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [42]:
with pd.ExcelWriter(f"{folder}/TRPV{trpv}_full_dataset_co25.xlsx", engine='openpyxl') as writer:
    for lipid, dataset in lipid_datasets.items():
        dataset.to_excel(writer, sheet_name=lipid)
    

In [43]:
lipid_datasets["CHOL"]

,Residue,Residue ID,Duration,Duration std,Occupancy,Occupancy std,Lipid Count,Lipid Count std,Koff,Residence Time,...,Binding Site R Squared Bootstrap avg,Binding Site Pose RMSD,Binding Site Surface Area,Lipid,aa,HI,Weighted HI,pdb_resnum,Binding Site Number aa,GRAVY_score
388,389MET,388,0.045000,0.058500,6.653360,2.024634,1.016463,0.011644,13.258048,0.075426,...,0.939687,0.547642,26.416307,CHOL,MET,1.9,0.028662,503,53.0,0.348701
389,390LYS,389,0.050000,0.055880,3.060546,1.787762,1.125000,0.148293,8.078618,0.123784,...,0.939687,0.547642,26.416307,CHOL,LYS,-3.9,-0.096551,504,53.0,0.348701
391,392LEU,391,0.059268,0.112454,6.054558,3.590349,1.000000,0.000000,4.810767,0.207867,...,0.939687,0.547642,26.416307,CHOL,LEU,3.8,0.157979,506,53.0,0.348701
392,393PHE,392,0.071972,0.116904,24.151697,13.614862,1.028798,0.029563,6.113654,0.163568,...,0.939687,0.547642,26.416307,CHOL,PHE,2.8,0.091598,507,53.0,0.348701
393,394VAL,393,0.020000,0.020702,0.332668,0.470464,0.333333,0.471405,30.439174,0.032852,...,0.939687,0.547642,26.416307,CHOL,VAL,4.2,0.027596,508,53.0,0.348701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,477PHE,2366,0.466923,0.540361,44.178310,12.287299,1.002398,0.003391,0.388676,2.572837,...,0.879212,0.383740,10.174164,CHOL,PHE,2.8,1.440789,591,27.0,0.518704
2413,524LEU,2413,0.156957,0.176030,4.324684,5.975443,0.666667,0.471405,3.397993,0.294291,...,0.879212,0.383740,10.174164,CHOL,LEU,3.8,0.223661,663,27.0,0.518704
2416,527ALA,2416,0.266667,0.474295,18.829009,5.627500,1.004695,0.006640,0.827514,1.208439,...,0.879212,0.383740,10.174164,CHOL,ALA,1.8,0.435038,666,27.0,0.518704
2420,531LEU,2420,0.762353,1.182284,14.903526,2.063614,1.000000,0.000000,0.968838,1.032164,...,0.879212,0.383740,10.174164,CHOL,LEU,3.8,0.784445,670,27.0,0.518704
